In [1]:
from PIL import Image
from glob import glob
import os 
import numpy as np
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense, Flatten,GlobalAveragePooling2D,Dropout
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet_v2 import MobileNetV2
import matplotlib.pyplot as plt 
import pickle

In [2]:
# Get a list of all the image file paths in a directory
data = []
labels = []
root = "data/"
categories = os.listdir(root)
for c in range(len(categories)):
    category = categories[c]
    path = os.path.join(root,category)
    image_files = glob(path+'/*.jpg')
    # Loop through the file paths and open each image
    for file in image_files:
        image = Image.open(file).convert('RGB')
        # Resize the image using the Image.resize() method
        image = image.resize((224, 224))
        data.append(np.array(image)/255)
        labels.append(c)

In [3]:
X = np.array(data)
y = np.array(labels)

In [4]:
categories

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [5]:
with open('categories.pickle','wb') as f : 
    pickle.dump(categories,f)

In [6]:
n = len(set(y))

In [7]:
X_train , X_test , y_train ,y_test = train_test_split(X,y,random_state=42)

In [8]:
base_model = MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

model = tf.keras.Sequential([
                              base_model,
                              GlobalAveragePooling2D(),
                              Dense(1024, activation = 'relu'),
                              Dropout(rate = 0.2),
                              Dense(n, activation='softmax')
                            ])

model.compile(optimizer = "adam",
                loss = "sparse_categorical_crossentropy",
                metrics=["accuracy"])

In [11]:
history = model.fit(X_train,y_train,epochs=20,validation_data=(X_test,y_test))

Epoch 1/20
204/204 [==============================] - 126s 615ms/step - loss: 1.4852 - accuracy: 0.4272 - val_loss: 1.5095 - val_accuracy: 0.3946
Epoch 2/20
204/204 [==============================] - 130s 637ms/step - loss: 1.3890 - accuracy: 0.4720 - val_loss: 1.5326 - val_accuracy: 0.4263
Epoch 3/20
204/204 [==============================] - 121s 596ms/step - loss: 1.3344 - accuracy: 0.4930 - val_loss: 1.4463 - val_accuracy: 0.4479
Epoch 4/20
204/204 [==============================] - 122s 598ms/step - loss: 1.2596 - accuracy: 0.5187 - val_loss: 1.4613 - val_accuracy: 0.4405
Epoch 5/20
204/204 [==============================] - 126s 618ms/step - loss: 1.1890 - accuracy: 0.5473 - val_loss: 1.5201 - val_accuracy: 0.4088
Epoch 6/20
204/204 [==============================] - 119s 583ms/step - loss: 1.1142 - accuracy: 0.5807 - val_loss: 1.4977 - val_accuracy: 0.4373
Epoch 7/20
204/204 [==============================] - 116s 571ms/step - loss: 1.0474 - accuracy: 0.6080 - val_loss: 1.4924 -

In [12]:
model.save("model.h5")